### Import functions

In [ ]:
import scipy
from os import listdir
import os
from os.path import isfile, join
import PIL
from PIL import Image
import pydicom
import math 
from pathlib import Path
import copy
import operator
import imageio
import matplotlib.pyplot as plt
import pandas as pd
import csv
import cv2
import tensorflow as tf
import keras
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import regularizers
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np
import glob
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam# use ADAM if it doesnt work
from tensorflow.keras.optimizers import legacy
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import MaxPooling2D
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

### Define functions to import data from folder `IM1`

In [ ]:
def split_patients(subjects, ratios):

    number_of_patients = len(subjects)
    train_size = int(number_of_patients*ratios[0])
    test_size = number_of_patients - train_size

    val_size = int(train_size*ratios[1])
    train_size -= val_size

    return train_size, val_size, test_size


def process_subjects(list_of_names, table):
    pixel_data = []
    labels = []

    for name in list_of_names:
        matching_files = table.loc[table['name'] == name]

        for _, row in matching_files.iterrows():
            filename = row['filename']
            label = row['label']
            try:
                img = Image.open("/Shared/CipacProcessing/Projects/RF_ML_ENA1/IM1/"+f"{filename}").convert('L')  # Convert image to grayscale
            except FileNotFoundError:
                print(f"File not found: {filename}")
                continue
            
            if img.size == (1024, 1024):
                # Apply histogram equalization
                img = cv2.equalizeHist(np.array(img))
                # Apply gamma correction
                img = np.array(255 * (img / 255) ** 0.7, dtype=np.uint8) # PLAY WITH THIS (0.85) - lower is lighter image
                pixel_array = np.array(img)               
                #min-max normalization
                min_value = np.min(pixel_array)
                max_value = np.max(pixel_array)
                pixel_array = (pixel_array - min_value) / (max_value - min_value + 1e-12)
                
                pixel_data.append(pixel_array.reshape(pixel_array.shape[0], pixel_array.shape[1], 1))
                labels.append(label)

    labels_array = np.array([0 if label.lower() == "oc" else 1 for label in labels])
    pixel_array = np.stack(pixel_data, axis=0)  # Stack arrays along a new dimension
    
    return pixel_array, labels_array

In [ ]:
list_files = glob.glob("/Shared/CipacProcessing/Projects/RF_ML_ENA1/IM1/*.jpg")
print(len(list_files))
table_metadata = pd.read_csv("/Shared/CipacProcessing/Projects/RF_ML_ENA1/table.csv")

### Split data into Train/Val/Test

In [ ]:
subjects = set()

for s in table_metadata["name"]:
    subjects.add(s)

print("Number of patients:", len(subjects))

np.random.seed(42) #Random seed, select if you want to keep same randomization
subject_list = list(subjects)
np.random.shuffle(subject_list)

ratios = [0.8, 0.2]#train/val
train_size, val_size, test_size = split_patients(subjects, ratios)

subject_train_set = subject_list[:train_size]
subject_val_set = subject_list[train_size:train_size+val_size]
subject_test_set = subject_list[train_size+val_size:]

print("Train size:", train_size)
print("Val size:", val_size)
print("Test size:", test_size)

### Get the respective Train/Val/Test `pixel arrays` and `labels`

In [ ]:
pixel_array_train, labels_train = process_subjects(subject_train_set, table_metadata)
pixel_array_val, labels_val = process_subjects(subject_val_set, table_metadata)
pixel_array_test, labels_test = process_subjects(subject_test_set, table_metadata)

# Validate the shape size
print(pixel_array_train.shape, labels_train.shape)
print(pixel_array_val.shape, labels_val.shape)
print(pixel_array_test.shape, labels_test.shape)

### Cell to print out images to see what they look like

In [ ]:
plt.subplot(1,2,1)
plt.title(labels_train[22])
plt.imshow(pixel_array_train[22], cmap="gray")
plt.axis("off")
plt.subplot(1,2,2)
plt.title(labels_train[23])
plt.imshow(pixel_array_train[23], cmap="gray")
plt.axis("off")
plt.show()

### Data Augmentation for Train set

In [ ]:
datagen = ImageDataGenerator( # PLAY WITH THESE
    rotation_range=1,  # Rotate image by up to 10 degrees
    width_shift_range=0.1,  # Shift image horizontally by up to 5% of the image size
    height_shift_range=0.1,  # Shift image vertically by up to 5% of the image size
    zoom_range=0.1,  # Zoom in or out by up to 5%
    horizontal_flip=True,  # Flip image horizontally
    # vertical_flip=True,
    # fill_mode='nearest',  # Fill missing pixels with the nearest value
    # preprocessing_function=preprocess_input  # Preprocess input using ResNet50 preprocessing
)

# no augmentation for val set
datagen_val = ImageDataGenerator()

### Define the Model

In [ ]:
input_A = keras.layers.Input(shape=(1024, 1024, 1))

conv1 = Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu")(input_A)
pool1 = MaxPooling2D((2, 2))(conv1)

conv2 = Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu")(pool1)
pool2 = MaxPooling2D((2, 2))(conv2)

conv3 = Conv2D(128, kernel_size=(3, 3), padding="same", activation="relu")(pool2)
pool3 = MaxPooling2D((2, 2))(conv3)

#fourth layer added - play with this, can remove too if needed
conv4 = Conv2D(256, kernel_size=(3, 3), padding="same", activation="relu")(pool3)
pool4 = MaxPooling2D((2, 2))(conv4)

fc = keras.layers.Flatten()(pool4)
fc = keras.layers.Dense(128, activation="relu")(fc)
fc = tf.keras.layers.Dropout(0.55)(fc) #0.5 - PLAY WITH DROPOUT LAYER
fc = keras.layers.Dense(1, activation="sigmoid")(fc)

model = keras.Model(inputs=[input_A], outputs=[fc])

### Define callbacks and compile model

In [ ]:
opt = legacy.Adam(lr=1e-4) #1e-4 - CAN PLAY WITH LEARN RATE

# Define the callbacks
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='auto')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-8)

model_chck_point_path = "/Shared/CipacProcessing/Projects/RF_ML_ENA1/my_models/Final_models/m6.h5" # CHANGE FILE NAME for NEW MODEL
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_chck_point_path, mode = 'auto', monitor='val_loss',verbose=0, save_best_only=True)

model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

### Train model and save best model checkpoint

In [ ]:
batch_size_ = 52
hist = model.fit(datagen.flow(pixel_array_train, labels_train, batch_size=batch_size_, shuffle=True), 
                 steps_per_epoch=len(pixel_array_train) // batch_size_,
                 epochs=100, 
                 verbose=1, 
                 validation_data=datagen_val.flow(pixel_array_val, labels_val, batch_size=24),
                 callbacks=[checkpoint, reduce_lr,earlyStopping])

### Print Results

In [ ]:
model_load = keras.models.load_model(model_chck_point_path)
test_loss, test_acc = model_load.evaluate(pixel_array_test, labels_test, verbose=0)

y_pred = model_load.predict(pixel_array_test)
binary_predictions = np.round(y_pred)

f1 = f1_score(labels_test, binary_predictions)

accuracy = accuracy_score(labels_test, binary_predictions)

print("score using .evaluate():",test_acc)
print("accuracy score:",accuracy)
print("f1-score:",f1)
print("----------------------------------------------------------------")
report = classification_report(labels_test, binary_predictions)
print (report)
print("----------------------------------------------------------------")
matrix = confusion_matrix(labels_test, binary_predictions)
print (matrix)

### Plot Confusion Matrix

In [ ]:
class_names = ['occluded', 'recanalized']

cmd = ConfusionMatrixDisplay(matrix, display_labels=class_names)
cmd.plot()

### Plot loss function

In [ ]:
plt.figure(figsize=(4,5), dpi=150)
plt.grid('on')
plt.title('Loss Function')
plt.plot(hist.history['loss'], 'b', lw=2, alpha=0.7, label='Training')
plt.plot(hist.history['val_loss'], 'r', lw=2, alpha=0.7, label='Val')
plt.legend(loc="upper right")
plt.show()

### CROSS VAL - same steps as above repeated

In [ ]:
# full_pixel_array = np.concatenate((pixel_array_train, pixel_array_val, pixel_array_test), axis=0)
# full_label_array = np.concatenate((labels_train, labels_val, labels_test), axis=0)
# print(full_pixel_array.shape,full_label_array.shape)
full_pixel_array = np.concatenate((pixel_array_train, pixel_array_val), axis=0)
full_label_array = np.concatenate((labels_train, labels_val), axis=0)
print(full_pixel_array.shape,full_label_array.shape)
print(pixel_array_val.shape,labels_val.shape)
print(pixel_array_test.shape,labels_test.shape)

In [ ]:
num_splits = 5
pixel_array_splits = np.array_split(full_pixel_array, num_splits)
label_array_splits = np.array_split(full_label_array, num_splits)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

num_splits = 5
scores_acc_eval = []
scores_acc_acc = []
scores_f1 = []
classification_reports = []
confusion_matrices = []
history_list = []

# Change folder name as needed
model_folder = "/Shared/CipacProcessing/Projects/RF_ML_ENA1/my_models/Final_models"
model_basename = "best_mod"

for val_index in range(num_splits):
    # select one split as validation set
    val_pixel_array = pixel_array_splits[val_index]
    val_label_array = label_array_splits[val_index]

    # combine remaining splits into training set
    train_pixel_array = np.concatenate(
        [split for i, split in enumerate(pixel_array_splits) if i != val_index],
        axis=0
    )
    train_label_array = np.concatenate(
        [split for i, split in enumerate(label_array_splits) if i != val_index],
        axis=0
    )

    # Shuffle training set and corresponding indices
    train_shuffle_indices = np.random.permutation(len(train_pixel_array))
    train_pixel_array = train_pixel_array[train_shuffle_indices]
    train_label_array = train_label_array[train_shuffle_indices]

    # Shuffle validation set and corresponding indices
    val_shuffle_indices = np.random.permutation(len(val_pixel_array))
    val_pixel_array = val_pixel_array[val_shuffle_indices]
    val_label_array = val_label_array[val_shuffle_indices]

    # Shuffle test set and corresponding indices
    test_shuffle_indices = np.random.permutation(len(pixel_array_test))
    test_pixel_array = pixel_array_test[test_shuffle_indices]
    test_label_array = labels_test[test_shuffle_indices]

    # Define the data generators for this fold
    train_datagen = datagen.flow(train_pixel_array, train_label_array, batch_size=32, shuffle=True)
    val_datagen = datagen_val.flow(val_pixel_array, val_label_array, batch_size=32)
    
    opt = legacy.Adam(lr=3e-4)

    # Define the callbacks
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.15, patience=6, min_lr=1e-8)
    # model_chck_point_path = "/Shared/CipacProcessing/Projects/RF_ML_ENA1/my_models/mymodelfin5.h5"
    model_chck_point_path = f"{model_folder}{model_basename}_split{val_index+1}.h5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(model_chck_point_path, mode = 'auto', monitor='val_loss',verbose=0, save_best_only=True)

    # Define and compile the model
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
    
    # Train the model for this fold
    hist = model.fit(train_datagen, 
                     steps_per_epoch=len(train_datagen),
                     epochs=45, 
                     verbose=1, 
                     validation_data=val_datagen,
                     callbacks=[checkpoint, reduce_lr])
    
    history_list.append(hist)

    model_load = keras.models.load_model(model_chck_point_path)

    # Evaluate the model on the test set for this fold and store the accuracy score
    test_loss, test_acc = model_load.evaluate(test_pixel_array, test_label_array, verbose=0)
    scores_acc_eval.append(test_acc)

    y_pred = model_load.predict(test_pixel_array)
    binary_predictions = np.round(y_pred)

    f1 = f1_score(test_label_array, binary_predictions)
    scores_f1.append(f1)

    accuracy = accuracy_score(test_label_array, binary_predictions)
    scores_acc_acc.append(accuracy)

    report = classification_report(test_label_array, binary_predictions)
    classification_reports.append(report)

    matrix = confusion_matrix(test_label_array, binary_predictions)
    confusion_matrices.append(matrix)

In [ ]:
print("scores acc eval:",scores_acc_eval)
print("accuracy scores:",scores_acc_acc)
print("f1-scores:",scores_f1)

In [ ]:
print("------------------------ RUN 1 (fold 1) ------------------------")
print(classification_reports[0])
print("------------------------ RUN 2 (fold 2) ------------------------")
print(classification_reports[1])
print("------------------------ RUN 3 (fold 3) ------------------------")
print(classification_reports[2])
print("------------------------ RUN 4 (fold 4) ------------------------")
print(classification_reports[3])
print("------------------------ RUN 5 (fold 5) ------------------------")
print(classification_reports[4])

In [ ]:
class_names = ['occluded', 'recanalized']

print("------------------------ RUN 1 (fold 1) ------------------------")
cmd = ConfusionMatrixDisplay(confusion_matrices[0], display_labels=class_names)
cmd.plot()
print("------------------------ RUN 2 (fold 2) ------------------------")
cmd = ConfusionMatrixDisplay(confusion_matrices[1], display_labels=class_names)
cmd.plot()
print("------------------------ RUN 3 (fold 3) ------------------------")
cmd = ConfusionMatrixDisplay(confusion_matrices[2], display_labels=class_names)
cmd.plot()
print("------------------------ RUN 4 (fold 4) ------------------------")
cmd = ConfusionMatrixDisplay(confusion_matrices[3], display_labels=class_names)
cmd.plot()
print("------------------------ RUN 5 (fold 5) ------------------------")
cmd = ConfusionMatrixDisplay(confusion_matrices[4], display_labels=class_names)
cmd.plot()

In [ ]:
plt.figure(figsize=(4,5), dpi=150)
plt.grid('on')
plt.title('Loss Function')
plt.plot(history_list[0].history['loss'], 'b', lw=2, alpha=0.7, label='Training')
plt.plot(history_list[0].history['val_loss'], 'r', lw=2, alpha=0.7, label='Val')
plt.legend(loc="upper right")

plt.show()

plt.figure(figsize=(4,5), dpi=150)
plt.grid('on')
plt.title('Loss Function')
plt.plot(history_list[1].history['loss'], 'b', lw=2, alpha=0.7, label='Training')
plt.plot(history_list[1].history['val_loss'], 'r', lw=2, alpha=0.7, label='Val')
plt.legend(loc="upper right")

plt.show()

plt.figure(figsize=(4,5), dpi=150)
plt.grid('on')
plt.title('Loss Function')
plt.plot(history_list[2].history['loss'], 'b', lw=2, alpha=0.7, label='Training')
plt.plot(history_list[2].history['val_loss'], 'r', lw=2, alpha=0.7, label='Val')
plt.legend(loc="upper right")

plt.show()

plt.figure(figsize=(4,5), dpi=150)
plt.grid('on')
plt.title('Loss Function')
plt.plot(history_list[3].history['loss'], 'b', lw=2, alpha=0.7, label='Training')
plt.plot(history_list[3].history['val_loss'], 'r', lw=2, alpha=0.7, label='Val')
plt.legend(loc="upper right")

plt.show()

plt.figure(figsize=(4,5), dpi=150)
plt.grid('on')
plt.title('Loss Function')
plt.plot(history_list[4].history['loss'], 'b', lw=2, alpha=0.7, label='Training')
plt.plot(history_list[4].history['val_loss'], 'r', lw=2, alpha=0.7, label='Val')
plt.legend(loc="upper right")

plt.show()